In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

--2024-07-04 10:41:59--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   103MB/s    in 0.2s    

2024-07-04 10:42:00 (103 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Filtering users with less than 200 ratings and books with less than 100 ratings
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

df_ratings_filtered = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(book_counts[book_counts >= 100].index)]

In [ ]:
# Create pivot table of ratings
pivot_ratings = df_ratings_filtered.pivot(index='isbn', columns='user', values='rating').fillna(0)

# Convert pivot table to sparse matrix
matrix = csr_matrix(pivot_ratings.values)

# Fit nearest neighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
def get_recommends(book_title):
    try:
        # Find the ISBN of the book title
        book_isbn = df_books[df_books['title'] == book_title]['isbn'].values[0]

        # Find the index of the book in the pivot table
        book_idx = pivot_ratings.index.get_loc(book_isbn)
    except (IndexError, KeyError):
        print(f"Book '{book_title}' not found in the dataset or doesn't have enough ratings.")
        return None

    # Get distances and indices of nearest neighbors
    distances, indices = model_knn.kneighbors(matrix[book_idx].reshape(1, -1), n_neighbors=6)

    # Initialize list to store recommended books
    recommended_books = []

    # Loop through the recommended indices and distances
    for i in range(len(distances.flatten()) - 1, 0, -1):
        isbn = pivot_ratings.index[indices.flatten()[i]]
        title = df_books[df_books['isbn'] == isbn]['title'].values[0]
        recommended_books.append([title, distances.flatten()[i]])

    return [book_title, recommended_books]

In [ ]:
books = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(books)

['The Queen of the Damned (Vampire Chronicles (Paperback))', [['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384], ['Interview with the Vampire', 0.73450685], ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566], ['Catch 22', 0.7939835]]]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
